In [1]:
import pandas as pd
import numpy as np

fake = pd.read_csv("C:\\Users\\sridhish\\Downloads\\archive\\Fake.csv")
real = pd.read_csv("C:\\Users\\sridhish\\Downloads\\archive\\True.csv")

In [2]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [3]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [30]:
import pandas as pd

fake = pd.read_csv("C:\\Users\\sridhish\\Downloads\\archive\\Fake.csv")
real = pd.read_csv("C:\\Users\\sridhish\\Downloads\\archive\\True.csv")
fake["Label"] = 0
real["Label"] = 1
df = pd.concat([fake,real])
df = df.sample(frac=1).reset_index(drop = True) # shuffle the data
df.tail()

,title,text,subject,date,Label
44893,Justice Dept. Has Had Enough — Sues Ferguson ...,The Department of Justice laid down the law on...,News,"February 11, 2016",0
44894,"In coastal New Jersey, a flood of criticism fo...",(Reuters) - Residents of New Jersey’s southern...,politicsNews,"January 25, 2016",1
44895,Republicans ask Clinton Foundation to produce ...,NEW YORK (Reuters) - The Republican National C...,politicsNews,"August 29, 2016",1
44896,BAM! THREE THINGS UFC’s Dana White Respects Ab...,"Dana White, Ultimate Fighting Championship Pre...",politics,"Jul 20, 2016",0
44897,NO PASSPORT NECESSARY: Tunnels Made By Drug Ca...,Mexicans have been given a green light by Obam...,left-news,"Jun 27, 2015",0


In [5]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text=re.sub(r'[^a-zA-Z]',' ',text.lower())
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)
df['cleaned_text'] = df['text'].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sridhish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sridhish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['cleaned_text'])
y = df['Label']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

Accuracy: 0.9875278396436525
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4772
           1       0.98      0.99      0.99      4208

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [8]:
import joblib

joblib.dump(tfidf, 'vectorizer.pkl')
joblib.dump(model, 'model.pkl')

['model.pkl']

In [9]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [38]:
import re
import string

def preprocess(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [44]:
loaded_model = joblib.load('model.pkl')
loaded_vectorizer = joblib.load('vectorizer.pkl')

sample_text = ["fifa 2018 world cup won by france"]
sample_cleaned = preprocess(sample_text[0])
sample_vector = loaded_vectorizer.transform([sample_cleaned])
print("Prediction:", loaded_model.predict(sample_vector))


Prediction: [0]


In [22]:
print(df['Label'].value_counts())

Label
0    23481
1    21417
Name: count, dtype: int64
